link for refreance - https://realpython.com/build-llm-rag-chatbot-with-langchain/

In [1]:
#pip install langchain==0.1.0 openai==1.7.2 langchain-openai==0.0.2 langchain-community==0.0.12 langchainhub==0.1.14

In [6]:
#pip install python-dotenv
import sys 
print(sys.executable)
import polars as pd
print(pd.__version__)


/home/codespace/.python/current/bin/python
1.4.0


In [3]:
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain_intro.chatbot import chat_model

In [4]:
messages = [SystemMessage(content="""You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions."""), 
HumanMessage(content="What is Medicaid managed care?")]

In [6]:
#chat_model.invoke(messages)
'''AIMessage(content='Medicaid managed care is a healthcare delivery system
in which states contract with managed care organizations (MCOs) to provide
healthcare services to Medicaid beneficiaries. Under this system, MCOs are
responsible for coordinating and delivering healthcare services to enrollees,
including primary care, specialty care, hospital services, and prescription
drugs. Medicaid managed care aims to improve care coordination, control costs,
and enhance the quality of care for Medicaid beneficiaries.')'''

"AIMessage(content='Medicaid managed care is a healthcare delivery system\nin which states contract with managed care organizations (MCOs) to provide\nhealthcare services to Medicaid beneficiaries. Under this system, MCOs are\nresponsible for coordinating and delivering healthcare services to enrollees,\nincluding primary care, specialty care, hospital services, and prescription\ndrugs. Medicaid managed care aims to improve care coordination, control costs,\nand enhance the quality of care for Medicaid beneficiaries.')"

In [7]:
#messages = [SystemMessage(content="""You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions."""), HumanMessage(content="How do I change a tire?")]
#chat_model.invoke(messages)
'''AIMessage(content='I apologize, but I can only provide assistance
and answer questions related to healthcare.')'''

"AIMessage(content='I apologize, but I can only provide assistance\nand answer questions related to healthcare.')"

In [32]:
from langchain.prompts import ChatPromptTemplate
review_template_str = """Your job is to use patient reviews to answer questions about 
their experience at a hospital. Use the following context to answer questions. 
Be as detailed as possible,  but don't make up any information that's not from the context.
 If you don't know an answer, say you don't know. \n{context} \n{question}"""

In [33]:
review_template = ChatPromptTemplate.from_template(review_template_str)
review_template

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="Your job is to use patient reviews to answer questions about \ntheir experience at a hospital. Use the following context to answer questions. \nBe as detailed as possible,  but don't make up any information that's not from the context.\n If you don't know an answer, say you don't know. \n{context} \n{question}"))])

In [34]:
context = "I had a great stay!"
question = "Did anyone have a positive experience?"


In [35]:
review_template.format(context=context, question=question)

"Human: Your job is to use patient reviews to answer questions about \ntheir experience at a hospital. Use the following context to answer questions. \nBe as detailed as possible,  but don't make up any information that's not from the context.\n If you don't know an answer, say you don't know. \nI had a great stay! \nDid anyone have a positive experience?"

In [47]:
cd ../

/workspaces/chat-hospital


/home/codespace/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from langchain_intro.chatbot import review_chain
context = "I had a great stay!"
question = "Did anyone have a positive experience?"


In [4]:
#review_chain.invoke({"context": context, "question": question})
'''AIMessage(content='Yes, the patient had a great stay and had a
positive experience at the hospital.')'''

"AIMessage(content='Yes, the patient had a great stay and had a\npositive experience at the hospital.')"

In [14]:
import polars as pl

HOSPITAL_DATA_PATH = "data/hospitals.csv"
data_hospitals = pl.read_csv(HOSPITAL_DATA_PATH)

print(data_hospitals.shape)
data_hospitals.head()




(30, 3)


hospital_id,hospital_name,hospital_state
i64,str,str
0,"""Wallace-Hamilton""","""CO"""
1,"""Burke, Griffin and Cooper""","""NC"""
2,"""Walton LLC""","""FL"""
3,"""Garcia Ltd""","""NC"""
4,"""Jones, Brown and Murray""","""NC"""


In [15]:
PHYSICIAN_DATA_PATH = "data/physicians.csv"
data_physician = pl.read_csv(PHYSICIAN_DATA_PATH)

print(data_physician.shape)


data_physician.head()

(500, 6)


physician_name,physician_id,physician_dob,physician_grad_year,medical_school,salary
str,i64,str,str,str,f64
"""Joseph Johnson""",0,"""1970-02-22""","""2000-02-22""","""Johns Hopkins University Schoo…",309534.155076
"""Jason Williams""",1,"""1982-12-22""","""2012-12-22""","""Mayo Clinic Alix School of Med…",281114.503559
"""Jesse Gordon""",2,"""1959-06-03""","""1989-06-03""","""David Geffen School of Medicin…",305845.584636
"""Heather Smith""",3,"""1965-06-15""","""1995-06-15""","""NYU Grossman Medical School""",295239.766689
"""Kayla Hunter DDS""",4,"""1978-10-19""","""2008-10-19""","""David Geffen School of Medicin…",298751.355201


In [19]:
PATH = "data/payers.csv"    # insurance companies
data = pl.read_csv(PATH)
print(data.shape)
data.head()

(5, 2)


payer_name,payer_id
str,i64
"""Medicaid""",0
"""UnitedHealthcare""",1
"""Aetna""",2
"""Cigna""",3
"""Blue Cross""",4


In [20]:
PATH = "data/reviews.csv"    # insurance companies
data = pl.read_csv(PATH)
print(data.shape)
data.head()

(1005, 6)


review_id,visit_id,review,physician_name,hospital_name,patient_name
i64,i64,str,str,str,str
0,6997,"""The medical staff at the hospi…","""Laura Brown""","""Wallace-Hamilton""","""Christy Johnson"""
9,8138,"""The hospital's commitment to p…","""Steven Watson""","""Wallace-Hamilton""","""Anna Frazier"""
11,680,"""The hospital's commitment to p…","""Chase Mcpherson Jr.""","""Wallace-Hamilton""","""Abigail Mitchell"""
892,9846,"""I had a positive experience ov…","""Jason Martinez""","""Wallace-Hamilton""","""Kimberly Rivas"""
822,7397,"""The medical team at the hospit…","""Chelsey Davis""","""Wallace-Hamilton""","""Catherine Yang"""


In [21]:
PATH = "data/visits.csv"    # insurance companies
data = pl.read_csv(PATH)
print(data.shape)
data.head()

(9998, 15)


patient_id,date_of_admission,billing_amount,room_number,admission_type,discharge_date,test_results,visit_id,physician_id,payer_id,hospital_id,chief_complaint,treatment_description,primary_diagnosis,visit_status
i64,str,f64,i64,str,str,str,i64,i64,i64,i64,str,str,str,str
0,"""2022-11-17""",37490.983364,146,"""Elective""","""2022-12-01""","""Inconclusive""",0,102,1,0,null,null,null,"""DISCHARGED"""
1,"""2023-06-01""",47304.064845,404,"""Emergency""",null,"""Normal""",1,435,4,5,null,null,null,"""OPEN"""
2,"""2019-01-09""",36874.896997,292,"""Emergency""","""2019-02-08""","""Normal""",2,348,2,6,null,null,null,"""DISCHARGED"""
3,"""2020-05-02""",23303.322092,480,"""Urgent""","""2020-05-03""","""Abnormal""",3,270,4,15,null,null,null,"""DISCHARGED"""
4,"""2021-07-09""",18086.344184,477,"""Urgent""","""2021-08-02""","""Normal""",4,106,2,29,"""Persistent cough and shortness…","""Prescribed a combination of in…","""J45.909 - Unspecified asthma, …","""DISCHARGED"""
